In [1]:
import sys
import os
import rasterio
import fiona
import numpy as np
import concurrent.futures

In [2]:
shpf = r"D:\Zach_Working\02_Scratch\PUG\HilliestBusRoute\Data\Random_Buffers.shp"
imgf = r"D:\Zach_Working\02_Scratch\PUG\HilliestBusRoute\Data\PDX_DEM_WGS84_UTM10N_Zm_INT16_ENVI"
test_out = imgf+"_test"

In [3]:
def scale(input_data):
    return input_data*3

num_workers = 8
with rasterio.Env():

    with rasterio.open(imgf) as src:

        # Create a destination dataset based on source params. The
        # destination will be tiled, and we'll process the tiles
        # concurrently.
        profile = src.profile
        profile.update(blockxsize=128, blockysize=128, tiled=True)

        with rasterio.open(test_out, "w", **profile) as dst:

            # Materialize a list of destination block windows
            # that we will use in several statements below.
            windows = [window for ij, window in dst.block_windows()]

            # This generator comprehension gives us raster data
            # arrays for each window. Later we will zip a mapping
            # of it with the windows list to get (window, result)
            # pairs.
            data_gen = (src.read(window=window) for window in windows[:2000])

            with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:

                # We map the compute() function over the raster
                # data generator, zip the resulting iterator with
                # the windows list, and as pairs come back we
                # write data to the destination dataset.
                for window, result in zip(windows, executor.map(scale, data_gen)):
                    dst.write(result, window=window)